In [1]:
!pip install pyarrow
!pip install wrds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 42.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
INFO: pip is looking at multiple versions of wrds to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.13
    Uninstalling SQLAlchemy-2.0.13:
      Successfully uninstalled SQLAlchemy-2.0.13

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import sys
import datetime as dt
import os
import pickle
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import numpy as np
import pandas as pd
import os
import sys
import pyarrow.parquet as pq
import re
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import wrds
import pytz
import math
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from tqdm import tqdm

In [3]:
def get_sd_and_ed_for_returns(date):
    start_current_month = date.replace(day=1)
    start_current_month = start_current_month + relativedelta(months = 1)
    start_next_month = start_current_month + relativedelta(months=1)
    #start_current_month = start_current_month + relativedelta(months=-2)

    start_current_month_str = start_current_month.strftime("%Y-%m-%d")
    start_next_month_str = start_next_month.strftime("%Y-%m-%d")

    return start_current_month_str, start_next_month_str

In [4]:
month_dict = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
folder_path = r'/data/workspace_files/section_dict'

In [5]:
def create_directories():
    folder_path = '/data/workspace_files/section_dict'
    for section in ['business', 'risk', 'qmr', 'mda']:
            for n_bucket in [5,10]:
                    for h in [3,6]:
                        for quartile in range(1,n_bucket + 1):
                            folder_path_req = f'{folder_path}/port_rets_n_scores/SP/{section}/h_{h}/n_{n_bucket}/Q{quartile}'
                            print(folder_path_req)
                            os.makedirs(folder_path_req, exist_ok = True)
                        folder_path_req = f'{folder_path}/port_rets_n_scores/SP/{section}/h_{h}/n_{n_bucket}/LS'
                        print(folder_path_req)
                        os.makedirs(folder_path_req, exist_ok = True)

In [6]:
create_directories()

/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_5/Q1
/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_5/Q2
/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_5/Q3
/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_5/Q4
/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_5/Q5
/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_5/LS
/data/workspace_files/section_dict/port_rets_n_scores/business/h_6/n_5/Q1
/data/workspace_files/section_dict/port_rets_n_scores/business/h_6/n_5/Q2
/data/workspace_files/section_dict/port_rets_n_scores/business/h_6/n_5/Q3
/data/workspace_files/section_dict/port_rets_n_scores/business/h_6/n_5/Q4
/data/workspace_files/section_dict/port_rets_n_scores/business/h_6/n_5/Q5
/data/workspace_files/section_dict/port_rets_n_scores/business/h_6/n_5/LS
/data/workspace_files/section_dict/port_rets_n_scores/business/h_3/n_10/Q1
/data/workspace_files/section_dict/po

In [12]:
def save_ls_portfolio(history, num_buckets, section, sim_score):
    folder_path = '/data/workspace_files/section_dict'
    for year in range(2012,2021):
        for month in range(1, 13):
            month_yr = f"{month_dict[month]}{year}"
            long_file_path = f'{folder_path}/port_rets_n_scores/{sim_score}/{section}/h_{history}/n_{num_buckets}/Q{num_buckets}/{month_yr}.parquet'
            print(long_file_path)
            short_file_path = f'{folder_path}/port_rets_n_scores/{sim_score}/{section}/h_{history}/n_{num_buckets}/Q1/{month_yr}.parquet'
            print(short_file_path)
            long_positions = pq.read_table(long_file_path).to_pandas()
            long_positions['pos_type'] = 1
            short_positions = pq.read_table(short_file_path).to_pandas()
            short_positions['pos_type'] = -1
            short_positions['weight'] = -1*short_positions['weight']
            positions = pd.concat([long_positions, short_positions])
            ls_file_path = f'{folder_path}/port_rets_n_scores/{sim_score}/{section}/h_{history}/n_{num_buckets}/LS/{month_yr}.parquet'
            print(ls_file_path)
            positions.to_parquet(ls_file_path)

In [13]:
for sim_score in ['cosine_similarity', 'jaccard_similarity']:
    for section in ['business', 'risk', 'qmr', 'mda']:
        for history in [3,6]:
            for n_bucket in [2,5]:
                save_ls_portfolio(history, n_bucket, section, sim_score)

/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/LS/Oct2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/Q2/Nov2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/Q1/Nov2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/LS/Nov2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/Q2/Dec2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/Q1/Dec2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/LS/Dec2016.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/Q2/Jan2017.parquet
/data/workspace_files/section_dict/port_rets_n_scores/cosine_similarity/business/h_6/n_2/Q1/Jan2017.parquet
/data/workspace_files/sectio

In [ ]:
: